In [1]:
import os
import json
import xmltodict
import numpy as np
import pandas as pd
from collections import defaultdict
source_file = './source/NL/NL.csv'

# North Brabant: NL.06, 2749990
# The Netherlands NL, 2750405



,geonameid,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
16,2743480,Zwingelspaan,Zwingelspaan,NaN,51.66000,4.49583,P,PPL,NL,NaN,06,1709,NaN,NaN,120,NaN,-2,Europe/Amsterdam,2017-03-24
25,2743489,Zwijnsbergen,Zwijnsbergen,NaN,51.52667,5.50694,P,PPL,NL,NaN,06,1948,NaN,NaN,0,NaN,13,Europe/Amsterdam,2017-08-16
26,2743490,Zwijnsbergen,Zwijnsbergen,NaN,51.64222,5.22194,S,EST,NL,NaN,06,0865,NaN,NaN,0,NaN,10,Europe/Amsterdam,2021-07-06
48,2743514,Zwartven,Zwartven,NaN,51.37390,5.10937,H,PND,NL,NaN,06,NaN,NaN,NaN,0,NaN,32,Europe/Amsterdam,2009-10-10
49,2743515,Zwartven,Zwartven,NaN,51.50849,4.73123,L,LCTY,NL,NaN,06,0879,NaN,NaN,0,NaN,8,Europe/Amsterdam,2017-08-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22807,12514717,Oss Elzenburg,Oss Elzenburg,NaN,51.78422,5.55320,R,RYD,NL,NaN,06,0828,NaN,NaN,0,NaN,7,Europe/Amsterdam,2023-03-06
22809,12519734,Wijk De Biezen,Wijk De Biezen,NaN,51.63196,4.94725,L,AREA,NL,NaN,06,0766,NaN,NaN,0,NaN,5,Europe/Amsterdam,2023-04-05
22821,12648437,Etten,Etten,NaN,51.57195,4.63256,P,PPLX,NL,NaN,06,0777,NaN,NaN,0,NaN,6,Europe/Amsterdam,2023-12-11
22833,12808958,Bastion Hotel Eindhoven,Bastion Hotel Eindhoven,NaN,51.40400,5.47621,S,HTL,NL,NaN,06,0866,NaN,NaN,0,NaN,20,Europe/Amsterdam,2024-05-14


In [ ]:
## ToDo

1. Add parent_ids

### Noord brabant is parent, no gemeente
for admin1_code == "06", admin2_code == Null, set parentid == geonamesid

### Noord brabant is parent, gemeente
for admin1_code == "06", admin2_code == "Admin2", set parentid == where feature_code = adm1

### gemeente is parent, 
for admin1_code == "06", "feature_code" == admin2, set parentid == where feature_code == "adm2" and admin2_code = admin2_code





2. Add columns from city500 (more admin and features)
3. Make a NB csv for alternative names

4. 

In [ ]:
geonames_df = pd.read_csv(source_file, delimiter=';', dtype={'geonameid': int, 'admin1_code': str, 'admin2_code': str, 'population': str})
geonames_df = geonames_df.loc[(geonames_df['country_code'] == 'NL') & (geonames_df['admin1_code'] == '06') | (geonames_df['geonameid'] == 2750405)]
geonames_df.to_csv('./out/geonames_nb.csv', index=False)
geonames_df


# Add parent_ids

In [8]:
# Initialize a parent_id column
geonames_df['parent_id'] = None

# Define a helper function to find the parent_id based on administrative codes
def find_parent_id(row, df):
    # Look for the parent in the same country with matching admin codes
    if pd.notna(row['admin2_code']):
        parent = df[(df['country_code'] == row['country_code']) & 
                    (df['admin1_code'] == row['admin1_code']) & 
                    (df['feature_code'].str.startswith('ADM2'))]
    #if pd.notna(row['admin1_code']):
    #    parent = df[(df['country_code'] == row['country_code']) & 
    #                (df['feature_code'].str.startswith('ADM1'))]
    #else:
    #    parent = pd.DataFrame()

    if not parent.empty:
        return parent.iloc[0]['geonameid'].astype(str)
    return None

# Apply the function to each row to generate the parent_id
geonames_df['parent_id'] = geonames_df.apply(find_parent_id, axis=1, df=geonames_df)
geonames_df.to_csv('./out/geonames_nb.csv', index=False)
geonames_df


UnboundLocalError: cannot access local variable 'parent' where it is not associated with a value

In [ ]:
unique_keys = set()

# Iterate over each dictionary in the list
for dictionary in records:
    # Add the keys of the current dictionary to the set
    unique_keys.update(dictionary['rdf:RDF']['schema:LandmarksOrHistoricalBuildings'].keys())

# Convert the set to a list (if needed)
all_keys_list = list(unique_keys)

print(json.dumps(all_keys_list, indent=2))

In [8]:
for record in records[:1]:
    print(record['@id'])
    for k,v in record['rdf:RDF']['schema:LandmarksOrHistoricalBuildings'].items():
        print(k,len(v), v)


NameError: name 'records' is not defined